## 1. Initialize settings

In [1]:
# change these to try this notebook out
BUCKET = 'cloudonair-ml-demo'
PROJECT = 'cloudonair-ml-demo'
REGION = 'us-central1'

In [2]:
import os

os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash

gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## 2. Explore Natality dataset

In [4]:
# Create SQL query using natality data after the year 2000
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 100
"""

In [5]:
# Call BigQuery and examine in dataframe
import pandas
from pandas.io import gbq

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'])
df.head()

Requesting query... ok.
Job ID: 680dc87d-7829-48bb-b908-72a8d55d9021
Query running...
Query done.
Processed: 5.2 GB Billed: 5.2 GB
Standard price: $0.03 USD

Retrieving results...
Got 100 rows.

Total time taken 1.82 s.
Finished at 2018-05-16 04:23:31.


,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.053894,False,15,1,38
1,7.685314,False,16,1,41
2,5.249206,False,17,1,36
3,8.562754,False,18,1,39
4,6.062712,False,19,1,39


## 3. Training on Cloud ML Engine

In [6]:
%%bash

if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
  # copy canonical set of preprocessed files if you didn't do previous notebook
  gsutil -m cp -R gs://cloud-training-demos/babyweight gs://${BUCKET}
fi

In [7]:
%%bash

gsutil ls gs://${BUCKET}/babyweight/preproc/*-00000*

gs://cloudonair-ml-demo/babyweight/preproc/eval.csv-00000-of-00012
gs://cloudonair-ml-demo/babyweight/preproc/train.csv-00000-of-00043


In [13]:
%%bash

OUTDIR=gs://${BUCKET}/babyweight/trained_model
JOBNAME=babyweight_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME

gsutil -m rm -rf $OUTDIR

gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$(pwd)/model_evaluation_pipeline/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=1.4 \
  -- \
  --bucket=${BUCKET} \
  --output_dir=${OUTDIR} \
  --train_examples=200000

gs://cloudonair-ml-demo/babyweight/trained_model us-central1 babyweight_180516_034734
jobId: babyweight_180516_034734
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [babyweight_180516_034734] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe babyweight_180516_034734

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs babyweight_180516_034734


## 3. Get a saved model directory

In [119]:
import datetime
import os
import pytz
import re
import urllib

from google.cloud import storage

def get_saved_model_dir(bucket, pattern='.*/babyweight/.*/saved_model.pb'):
  created_time = datetime.datetime(datetime.MINYEAR, 1, 1, tzinfo=pytz.timezone('UTC'))
  recent_model_path = None

  for blob in storage.Client().bucket(bucket).list_blobs():
    if p.match(urllib.unquote(blob.path)):
      if created_time < blob.time_created:
        recent_model_path = blob.path
        created_time = blob.time_created

  saved_model = urllib.unquote(recent_model_path.split('/')[-1])
  saved_model_dir = '/'.join(saved_model.split('/')[:-1])
  return 'gs://{0}/{1}'.format(os.environ['BUCKET'], saved_model_dir)

saved_model_dir = get_saved_model_dir(os.environ['BUCKET'])

if saved_model_dir:
  os.environ['SAVED_MODEL_DIR'] = saved_model_dir
else:
  print('wait until saved_model.pb is written by Cloud ML Engine...')

## 4. Testing an evaluation pipeline with DirectRunner

### Configure a DirectRunner job

In [111]:
import os
import datetime

os.environ['RUNNER'] = 'DirectRunner'
os.environ['JOB_NAME'] = 'evaluate-ml-model-{0}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
os.environ['OUTPUT_TABLE'] = 'cloudonair-ml-demo:model_evaluation.sample'

### Execute a DirectRunner job

In [112]:
%%bash

cd $(pwd)/model_evaluation_pipeline/evaluator

python -m main \
  --datasize=100 \
  --year_from=1974 \
  --year_to=1978 \
  --saved_model_dir=${SAVED_MODEL_DIR} \
  --output_table=${OUTPUT_TABLE} \
  --project=${PROJECT} \
  --runner=DirectRunner \
  --region=${REGION} \
  --job_name=${JOB_NAME}

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/optimize/_minimize.py:32: ImportWarning: Not importing directory '/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/optimize/lbfgsb': missing __init__.py
  from .lbfgsb import _minimize_lbfgsb
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/spatial/__init__.py:95: ImportWarning: Not importing directory '/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/spatial/qhull': missing __init__.py
  from .qhull import *
INFO:root:Running pipeline with DirectRunner.
/usr/local/envs/py2env/lib/python2.7/site-packages/oauth2client/contrib/gce.py:99: UserWarning: You have requested explicit scopes to be used wi

### Show results of a DirectRunner Job

In [113]:
import pandas
from pandas.io import gbq

query = """
SELECT * FROM `{0}` LIMIT 10
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'])
df.head()

Requesting query... ok.
Job ID: 87f13e91-2d91-4cfb-be75-dad41e33a1c5
Query running...
Query done.
Processed: 164.9 KB Billed: 10.0 MB
Standard price: $0.00 USD

Retrieving results...
Got 10 rows.

Total time taken 1.55 s.
Finished at 2018-05-16 05:23:54.


,is_male,mother_age,plurality,gestation_weeks,weight_true,weight_predicted,weight_residual,model,testdata,time_inference
0,True,19,Single(1),41,7.936641,0.295066,7.641575,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.957012
1,True,27,Single(1),99,6.250105,0.285623,5.964482,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.888109
2,True,28,Single(1),42,6.499227,0.293240,6.205988,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.890970
3,False,19,Single(1),99,7.813183,0.285276,7.527906,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.894070
4,False,18,Single(1),99,6.750554,0.285276,6.465278,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,0.912189


In [114]:
import pandas
from pandas.io import gbq

query = """
SELECT
  model,
  testdata,
  SQRT(SUM(POW(weight_residual,2))/COUNT(*)) AS RMSE,
  AVG(time_inference) AS AVG_MSEC
FROM
  `{0}`
GROUP BY
  model, testdata
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'])
df.head()

Requesting query... ok.
Job ID: 47a45702-4d69-408f-b2d5-010f2eb5a64a
Query running...
Query done.
Processed: 111.7 KB Billed: 10.0 MB
Standard price: $0.00 USD

Retrieving results...
Got 5 rows.

Total time taken 1.56 s.
Finished at 2018-05-16 05:24:04.


,model,testdata,RMSE,AVG_SEC
0,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,7.099255,2.889484
1,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,7.097674,4.989076
2,gs://cloudonair-ml-demo/babyweight/trained_mod...,1984-1988,7.208219,3.528221
3,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,11.089627,2.996402
4,gs://cloudonair-ml-demo/babyweight/trained_mod...,1974-1978,1.318639,2.993131


## 5. Running an evaluation pipeline with DataflowRunner

### Configure a Dataflow job

In [115]:
import os
import datetime

os.environ['RUNNER'] = 'DataflowRunner'
os.environ['STAGING_LOCATION'] = 'gs://{0}/babyweight/staging'.format(os.environ['BUCKET'])
os.environ['TEMP_LOCATION'] = 'gs://{0}/babyweight/temp'.format(os.environ['BUCKET'])
os.environ['JOB_NAME'] = 'evaluate-ml-model-{0}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
os.environ['OUTPUT_TABLE'] = 'cloudonair-ml-demo:model_evaluation.prediction_results'

### Execute a Dataflow job

In [116]:
%%bash

cd $(pwd)/model_evaluation_pipeline/evaluator

python -m main \
  --datasize=1000000 \
  --year_from=1984 \
  --year_to=1988 \
  --saved_model_dir=${SAVED_MODEL_DIR} \
  --output_table=${OUTPUT_TABLE} \
  --project=${PROJECT} \
  --runner=${RUNNER} \
  --region=${REGION} \
  --staging_location=${STAGING_LOCATION} \
  --temp_location=${TEMP_LOCATION} \
  --job_name=${JOB_NAME} \
  --setup_file=$(pwd)/setup.py

running sdist
running egg_info
writing requirements to model_evaluation_demo.egg-info/requires.txt
writing model_evaluation_demo.egg-info/PKG-INFO
writing top-level names to model_evaluation_demo.egg-info/top_level.txt
writing dependency_links to model_evaluation_demo.egg-info/dependency_links.txt
reading manifest file 'model_evaluation_demo.egg-info/SOURCES.txt'
writing manifest file 'model_evaluation_demo.egg-info/SOURCES.txt'
running check
creating model-evaluation-demo-0.1
creating model-evaluation-demo-0.1/model_evaluation_demo.egg-info
creating model-evaluation-demo-0.1/process
copying files to model-evaluation-demo-0.1...
copying setup.py -> model-evaluation-demo-0.1
copying model_evaluation_demo.egg-info/PKG-INFO -> model-evaluation-demo-0.1/model_evaluation_demo.egg-info
copying model_evaluation_demo.egg-info/SOURCES.txt -> model-evaluation-demo-0.1/model_evaluation_demo.egg-info
copying model_evaluation_demo.egg-info/dependency_links.txt -> model-evaluation-demo-0.1/model_eva

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/optimize/_minimize.py:32: ImportWarning: Not importing directory '/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/optimize/lbfgsb': missing __init__.py
  from .lbfgsb import _minimize_lbfgsb
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/spatial/__init__.py:95: ImportWarning: Not importing directory '/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/spatial/qhull': missing __init__.py
  from .qhull import *
/usr/local/envs/py2env/lib/python2.7/site-packages/apache_beam/coders/typecoders.py:135: UserWarning: Using fallback coder for typehint: Dict[Any, Any].
  warnings.warn('Using fallback coder

In [120]:
query = """
SELECT
  model,
  testdata,
  SQRT(SUM(POW(weight_residual,2))/COUNT(*)) AS RMSE,
  AVG(time_inference) AS AVG_MSEC
FROM
  `{0}`
GROUP BY
  model, testdata
""".format(os.environ['OUTPUT_TABLE'].replace(':', '.'))

df = gbq.read_gbq(query=query, dialect='standard', project_id=os.environ['PROJECT'])
df.head()

Requesting query... ok.
Job ID: 961fd6b7-a276-4095-87f3-0d4d0ff5c502
Query running...
Query done.
Processed: 60.9 KB Billed: 10.0 MB
Standard price: $0.00 USD

Retrieving results...
Got 4 rows.

Total time taken 1.62 s.
Finished at 2018-05-16 05:30:33.


,model,testdata,RMSE,AVG_MSEC
0,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,7.049755,2.996565
1,gs://cloudonair-ml-demo/babyweight/trained_mod...,1969-1973,11.265065,3.257217
2,gs://cloudonair-ml-demo/babyweight/trained_mod...,1984-1988,7.202314,3.076031
3,gs://cloudonair-ml-demo/babyweight/trained_mod...,1989-1993,6.882426,1.946766


## 6. Visualize Optimizing and Satisficing Metric with DataStudio

<img src="model_evaluation_pipeline/dashboard1.png">

<img src="model_evaluation_pipeline/dashboard2.png">